getting ready
https://xgboost.readthedocs.io/en/latest/python/python_intro.html#training

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost import XGBRegressor
from xgboost import plot_importance
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import os

import time

In [2]:
test = pd.read_pickle('./data/prepro2_2test.pkl')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6426000 entries, 0 to 6425999
Data columns (total 13 columns):
 #   Column            Dtype
---  ------            -----
 0   year              int16
 1   month             int16
 2   day               int16
 3   dayofweek         int16
 4   shop_id           int16
 5   item_id           int16
 6   item_name         int16
 7   item_category_id  int16
 8   main_cat          int16
 9   sub_cat           int16
 10  dayofyear         int16
 11  dayofmonth        int16
 12  weekofyear        int16
dtypes: int16(13)
memory usage: 208.4 MB


In [4]:
train = pd.read_pickle('./data/prepro2_2train.pkl')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70512330 entries, 0 to 70512329
Data columns (total 15 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   year              int16         
 2   month             int16         
 3   day               int16         
 4   dayofweek         int16         
 5   shop_id           int16         
 6   item_id           int16         
 7   item_name         int16         
 8   item_category_id  int16         
 9   main_cat          int16         
 10  sub_cat           int16         
 11  item_cnt_day      int16         
 12  dayofyear         int16         
 13  dayofmonth        int16         
 14  weekofyear        int16         
dtypes: datetime64[ns](1), int16(14)
memory usage: 2.9 GB


In [5]:
train.head()

,date,year,month,day,dayofweek,shop_id,item_id,item_name,item_category_id,main_cat,sub_cat,item_cnt_day,dayofyear,dayofmonth,weekofyear
0,2013-01-02,0,1,2,2,59,22154,18108,37,11,1,1,2,2,1
1,2013-01-02,0,1,2,2,59,2565,2151,56,13,5,0,2,2,1
2,2013-01-02,0,1,2,2,59,2546,2150,55,13,4,0,2,2,1
3,2013-01-02,0,1,2,2,59,2715,2035,19,5,12,0,2,2,1
4,2013-01-02,0,1,2,2,59,2716,2035,28,8,34,0,2,2,1


xgboost default with dataw without 0 value

In [6]:
train['datenum'] = train.dayofyear + train.year*365

In [7]:
train.head()

,date,year,month,day,dayofweek,shop_id,item_id,item_name,item_category_id,main_cat,sub_cat,item_cnt_day,dayofyear,dayofmonth,weekofyear,datenum
0,2013-01-02,0,1,2,2,59,22154,18108,37,11,1,1,2,2,1,2
1,2013-01-02,0,1,2,2,59,2565,2151,56,13,5,0,2,2,1,2
2,2013-01-02,0,1,2,2,59,2546,2150,55,13,4,0,2,2,1,2
3,2013-01-02,0,1,2,2,59,2715,2035,19,5,12,0,2,2,1,2
4,2013-01-02,0,1,2,2,59,2716,2035,28,8,34,0,2,2,1,2


In [8]:
test['datenum'] = test.dayofyear + test.year*365

In [9]:
test.head()

,year,month,day,dayofweek,shop_id,item_id,item_name,item_category_id,main_cat,sub_cat,dayofyear,dayofmonth,weekofyear,datenum
0,2,11,1,6,5,5037,4182,19,5,12,305,1,44,1035
1,2,11,1,6,5,5320,4275,55,13,4,305,1,44,1035
2,2,11,1,6,5,5233,4147,19,5,12,305,1,44,1035
3,2,11,1,6,5,5232,4147,23,5,18,305,1,44,1035
4,2,11,1,6,5,5268,4139,20,5,13,305,1,44,1035


In [10]:
train_datenum = train[['datenum', 'shop_id', 'item_id', 'item_name',
       'item_category_id', 'main_cat', 'sub_cat', 'item_cnt_day' ]]

In [14]:
train_datenum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70512330 entries, 0 to 70512329
Data columns (total 8 columns):
 #   Column            Dtype
---  ------            -----
 0   datenum           int16
 1   shop_id           int16
 2   item_id           int16
 3   item_name         int16
 4   item_category_id  int16
 5   main_cat          int16
 6   sub_cat           int16
 7   item_cnt_day      int16
dtypes: int16(8)
memory usage: 1.6 GB


In [12]:
test_datenum = test[['datenum', 'shop_id', 'item_id', 'item_name',
       'item_category_id', 'main_cat', 'sub_cat' ]]

In [15]:
test_datenum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6426000 entries, 0 to 6425999
Data columns (total 7 columns):
 #   Column            Dtype
---  ------            -----
 0   datenum           int16
 1   shop_id           int16
 2   item_id           int16
 3   item_name         int16
 4   item_category_id  int16
 5   main_cat          int16
 6   sub_cat           int16
dtypes: int16(7)
memory usage: 134.8 MB


In [16]:
train_datenum.to_pickle('./data/prepro3train.pkl')
test_datenum.to_pickle('./data/prepro3test.pkl')
